In [3]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
celtics_id = celtics['id']

In [7]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612738,BOS,Boston Celtics,0022400104,2024-10-28,BOS vs. MIL,W,242,119,...,0.789,4,34,38,24,5,5,12,18,11.0
1,22024,1610612738,BOS,Boston Celtics,0022400089,2024-10-26,BOS @ DET,W,239,124,...,0.867,13,24,37,19,10,6,10,16,6.0
2,22024,1610612738,BOS,Boston Celtics,0022400073,2024-10-24,BOS @ WAS,W,240,122,...,0.778,12,37,49,21,7,3,14,19,20.0
3,22024,1610612738,BOS,Boston Celtics,0022400061,2024-10-22,BOS vs. NYK,W,240,132,...,0.875,11,29,40,33,6,3,3,15,23.0
4,12024,1610612738,BOS,Boston Celtics,0012400050,2024-10-15,BOS @ TOR,L,239,118,...,0.857,8,28,36,19,10,5,6,15,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3921,21983,1610612738,BOS,Boston Celtics,0028300052,1983-11-05,BOS @ WAS,W,240,120,...,0.778,5,28,33,36,7,7,16,26,NaN
3922,21983,1610612738,BOS,Boston Celtics,0028300043,1983-11-04,BOS vs. IND,W,240,121,...,0.841,11,34,45,28,8,5,22,26,NaN
3923,21983,1610612738,BOS,Boston Celtics,0028300033,1983-11-02,BOS vs. MIL,W,240,119,...,0.816,14,24,38,26,7,7,18,18,NaN
3924,21983,1610612738,BOS,Boston Celtics,0028300019,1983-10-29,BOS @ CLE,W,240,108,...,0.690,13,31,44,25,12,7,13,22,NaN


In [29]:
games_2024 = games[(games['GAME_DATE'] >= '2023-10-24') & (games['GAME_DATE'] <= '2024-04-14')]
games_2024.to_excel("games_2024.xlsx", index=False)